In [2]:

#!pip install langchain_openai langchain_community pypdf chromadb

In [3]:

import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
import urllib.request
import gradio as gr

In [4]:
# OpenAI API 키 설정 (본인의 API 키로 변경)
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
API_KEY = os.getenv("OPENAI_API_KEY")

# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key


In [5]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")


('2020_경제금융용어 700선_게시.pdf', <http.client.HTTPMessage at 0x1156b8b50>)

In [61]:

loader = PyPDFLoader("2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()
print('문서의 수 :', len(texts))

문서의 수 : 366


In [62]:
print('첫번째 청크 출력 :', texts[0])
print('마지막 청크 출력 :', texts[-1])
print('마지막에서 두번째 청크 출력 :', texts[-2])
texts = texts[:-1]
print('마지막 데이터 제거 후 청크의 개수:', len(texts))

첫번째 청크 출력 : page_content='iii
찾아보기
한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 
의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 
개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . 
이의 일환으로 2018년 
 경제금융용어 700선
 을 발간하였는데 그간 동 책자에 대한 
수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .
지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 
업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 이해하는데 도움이 되는 
시사 경제금융 용어들을 수록하였습니다. 용어해설은 개념과 도입 배경, 의미, 적용 
사례 등을 담아 쉽게 이해할 수 있도록 하였습니다. 또한 e-book으로도 제작하여 
독자들의 편의성과 가독성을 높였습니다 . 
코로나 19로 이전에 경험하지 못한 경제위기를 겪으면서 경제·금융 현상에 대한 
일반인들의 관심도가 이전에 비해 높아졌습니다. 아무쪼록 이 책자가 한국은행의 주요 
정책과 국내외 경제를 이해하는 데 조금이나마 도움이 되기를 바라겠습니다. 끝으로 
경제금융용어 700선
  집필에 참여해주신 경제교육실 교수님들과 용어 집필에 좋은 
의견을 주신 관련부서 여러분께 감사드립니다 .
2020년 8월
한국은행 경제교육실장 박 철 원
머 리 말' metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subjec

In [63]:
# model =text-embedding-ada-002 --> 구형모델

embedding = OpenAIEmbeddings(chunk_size=100)


# # 소형 (저비용, 속도 빠름)
# emb_small = OpenAIEmbeddings(model="text-embedding-3-small", chunk_size=100)
# # 대형 (더 정밀)
# # emb_large = OpenAIEmbeddings(model="text-embedding-3-large")
# embedding = emb_small

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)

In [64]:
print(vectordb._collection.count())

717


In [35]:

for key in vectordb._collection.get():
  print(key)

ids
embeddings
documents
uris
included
data
metadatas


In [65]:

documents = vectordb._collection.get()['documents']
print('청크의 개수 :', len(documents))
print('-' * 50)
print('0번 청크 출력 :', documents[0])

청크의 개수 : 717
--------------------------------------------------
0번 청크 출력 : 1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [66]:
embeddings = vectordb._collection.get(include=['embeddings'])['embeddings']
print('임베딩 벡터의 개수 :', len(embeddings))

임베딩 벡터의 개수 : 717


In [67]:

print('0번 청크의 임베딩 값 출력 :', embeddings[0])
print('0번 청크의 임베딩 값의 길이 :', len(embeddings[0]))

0번 청크의 임베딩 값 출력 : [0.02719978 0.02220346 0.04836953 ... 0.00474058 0.05228905 0.01140323]
0번 청크의 임베딩 값의 길이 : 1536


In [68]:
metadatas = vectordb._collection.get()['metadatas']
print('metadatas의 개수 :', len(metadatas))
print('0번 청크의 출처 :', metadatas[0])

metadatas의 개수 : 717
0번 청크의 출처 : {'total_pages': 371, 'source': '2020_경제금융용어 700선_게시.pdf', 'moddate': 'January 31, 2018 16:21', 'page': 17, 'page_label': '18', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'producer': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'author': 'INSUNG DATA INC.', 'moddate--text': 'January 31, 2018 16:21', 'creator': 'Smart PDF Imposition 1.0', 'creationdate--text': 'January 31, 2018 16:21'}


In [69]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

docs = retriever.get_relevant_documents("가상통화 궁금해")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('두번째 유사 문서 :', docs[1])

유사 문서 개수 : 2
----------------------------------------
첫번째 유사 문서 : page_content='72
경제금융용어 700선
기축통화
기축통화란 여러 국가의 암묵적인 동의하에 국제거래에서 중심적인 역할을 하는 
통화를 지칭한다. 구체적으로는 ① 국제무역결제에 사용되는 통화 ② 환율 평가 시의 
지표가 되는 통화 ③ 대외준비자산으로 보유되는 통화 등의 의미를 포함한다. 그러므로 
어떤 나라의 통화가 기축통화가 되기 위해서는 세계적으로 원활히 유통될 수 있도록 
유동성이 풍부하여야 하고 거래당사자들이 믿고 사용할 수 있도록 신뢰성을 갖추어야 
하며, 국제적으로 경제력은 물론 정치력･군사력까지 인정받는 국가의 통화여야 한다. 
20세기 초반까지는 세계 금융경제의 중심이었던 영국의 파운드화가 기축통화로서 국제
거래에 주로 이용되었으며, 2차 세계대전 이후에는 전 세계 외환거래 및 외환보유액의 
상당 부분을 차지하는 미국 달러화가 기축통화로 인정받고 있다. 한편, 현재 국제거래에
서 비교적 자주 사용되는 통화로는 미국 달러화이외에도 유로화, 영국 파운드화, 일본 
엔화, 스위스 프랑화 등이 있는데 이들은 흔히 교환성통화라 한다 . 
 연관검색어 : 교환성 통화
기타기본자본(Additional Tier 1)
기타기본자본(AT1; Additional Tier 1)은 요건을 충족하는 기타기본자본증권 및 이와 
관련된 자본잉여금, 자회사가 발행한 기타기본자본증권의 제3자 보유분 중 일부 금액 
등으로 구성된다. 기타기본자본증권으로 인정받기 위해서는 예금자, 일반채권자 및 
후순위채권보다 후순위, 영구적, 배당 및 이자지급의 임의성, 중도상환 유인이 없을 
것 등 총 14개 요건을 충족해야 한다 .
기펜재
일반적으로 재화는 수요의 법칙에 따라 가격이 하락하면 수요량이 늘어난다. 그러나 
예외적으로 재화의 가격이 하락할 때 수요량이 감소하는 현상을 관찰할 수 있다. 이렇게 
가격이 하락할 경우 수요의 법칙에 위배되는 재화를 이를 처음 

In [70]:

template = """당신은 한국은행에서 만든 금융 용어를 설명해주는 금융해설자입니다.
주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

In [71]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [ ]:
# ============================
# 5. Ensemble Retriever (BM25+FAISS)
# ============================
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever # Corrected import

vector_retriever = vectordb.as_retriever(search_kwargs={"k": 30})

bm25 = BM25Retriever.from_documents(texts)
bm25.k = 30

ensemble_retriever = EnsembleRetriever(retrievers=[bm25, vector_retriever], weights=[0.4, 0.6])

docs = ensemble_retriever.invoke("인플레이션이 무엇인가요?")
print("\n🔹 Ensemble Retriever 결과:")
for d in docs:
    print("-", d.page_content[:120].replace("\n"," ",), d.metadata)

## ReRanker

In [96]:
from sentence_transformers import CrossEncoder
import torch

def get_device():
    # Determine best available device (MPS > CUDA > CPU)
    if torch.backends.mps.is_available():
        device = torch.device("mps")  # Mac M-series GPU
    elif torch.cuda.is_available():
        device = torch.device("cuda")  # NVIDIA GPU
    else:
        device = torch.device("cpu")   # CPU fallback
    return device

device = get_device()
print("Device:", device)

# 공개 모델 (한국어 포함 다국어에 강함)
# 더 강한 모델: "BAAI/bge-reranker-large" (속도/메모리↑)
reranker = CrossEncoder(
    "BAAI/bge-reranker-base",
    device=device,
    max_length=512
)

def cross_encoder_rerank(query, docs, top_k=5, batch_size=32):
    pairs = [(query, d.page_content) for d in docs]
    scores = []
    for i in range(0, len(pairs), batch_size):
        with torch.inference_mode():
            s = reranker.predict(pairs[i:i+batch_size])
        # numpy -> list
        scores.extend(s.tolist() if hasattr(s, "tolist") else list(s))
    scored = list(zip(docs, scores))
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:top_k]

# ============================================
# 6) 통합 질의 함수 (1차 검색 → 2차 재랭킹)
# ============================================
def ask(query, top_k_first=30, top_k_final=5, show_scores=True):
    candidates = ensemble_retriever.invoke(query)
    reranked = cross_encoder_rerank(query, candidates, top_k=top_k_final)
    print(f"\n🔎 Q: {query}")
    print(f"  1차 후보 {len(candidates)}개 → 2차 상위 {len(reranked)}개")

    print(f"  1차 후보 상위 {len(reranked)}개")
    for i, d in enumerate(candidates, 0):
        page = d.metadata.get("page", "?")
        preview = d.page_content[:160].replace("\n"," ")
        if show_scores:
            print(f"{i:>2}. p.{page} | {preview}...")
        else:
            print(f"{i:>2}. p.{page} | {preview}...")
        if i >= top_k_final:
            break

    print(f"  2차 후보 상위 {len(reranked)}개")
    for i, (d, s) in enumerate(reranked, 1):
        page = d.metadata.get("page", "?")
        preview = d.page_content[:160].replace("\n"," ")
        if show_scores:
            print(f"{i:>2}. p.{page} | score={s:.4f} | {preview}...")
        else:
            print(f"{i:>2}. p.{page} | {preview}...")



Device: mps


In [97]:

ask("가상화폐를 설명해줘")


🔎 Q: 가상화폐를 설명해줘
  1차 후보 58개 → 2차 상위 5개
  1차 후보 상위 5개
 0. p.133 | 117 ㅁ  따라 자국 통화의 가치가 수동적으로 변동하게 됨으로써 통화의 가치가 자국의 경제상황  또는 수출경쟁력을 적절히 반영되지 못하는 문제점이 발생할 수 있다 .   연관검색어 : 고정환율제도/자유변동환율제도, 복수통화바스켓제도 미발행화폐  미발행화폐란 한국은행이 보유하고 있는 은...
 1. p.136 | 120 경제금융용어 700선 발행중지화폐/유통정지화폐 발행중지화폐는 발권기관인 한국은행이 화폐의 원활한 유통 등을 위해 발행을 중지하 기로 한 화폐이다. 발행중지화폐는 비록 화폐 발행은 중단되지만 화폐로서의 강제통용력 도 있고 지급결제수단으로 사용도 가능하다. 또한 발행중지화폐는 한국은...
 2. p.345 | 329 ㅎ  화폐의 액면체계 화폐의 액면체계는 한 국가의 화폐를 구성하는 액면금액의 틀을 의미하는데, 보통  액면체계는 일상적인 거래 편의 도모라는 대원칙 아래 국가의 경제 ･사회적 여건과  국민의 화폐사용 습관의 영향을 받는다. 화폐의 액면체계는 대부분 기본수(1･5･10 등)에  1...
 3. p.266 | 250 경제금융용어 700선 전자화폐 IC카드 등에 화폐가치를 저장하였다가 상품 등의 구매에 사용할 수 있는 전자지급수단 으로서 범용성과 환금성을 갖춘 것을 말한다. 전자화폐(electronic money)의 유형은  가치저장 매체에 따라 IC카드형과 네트워크형으로 구분할 수 있다. I...
 4. p.344 | 328 경제금융용어 700선 현재 우리나라 은행권에는 오만원권에 띠형 홀로그램이, 만원과 오천원권에는 패치형  홀로그램이 각각 적용되어 있다. 오만원권의 띠형 홀로그램은 앞면 왼쪽 끝 부분에 부착되 어 있으며 보는 각도에 따라 상･중･하 3곳에서 ① 우리나라 지도 ② 태극 ③ 4괘 무늬...
 5. p.47 | 31 ㄱ  볼 수 있다. 그러나 좁은 의미로는 SDR 가치 산정에 사용되는 바스켓 통화의 요건에  

In [76]:
# ============================
# Cross-Encoder Reranker Retriever 생성
# ============================

from langchain.schema import BaseRetriever
from langchain.schema import Document
from typing import List

class CrossEncoderRetriever(BaseRetriever):
    def __init__(self, ensemble_retriever, reranker_func, top_k_first=30, top_k_final=5):
        super().__init__()
        self._ensemble_retriever = ensemble_retriever
        self._reranker_func = reranker_func
        self._top_k_first = top_k_first
        self._top_k_final = top_k_final
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        # 1차: EnsembleRetriever로 후보 문서 검색
        candidates = self._ensemble_retriever.invoke(query)
        
        # 2차: Cross-Encoder로 reranking
        reranked = self._reranker_func(query, candidates, top_k=self._top_k_final)
        
        # 문서만 반환 (점수 제거)
        return [doc for doc, score in reranked]

# CrossEncoderRetriever 인스턴스 생성
cross_encoder_retriever = CrossEncoderRetriever(
    ensemble_retriever=ensemble_retriever,
    reranker_func=cross_encoder_rerank,
    top_k_first=30,
    top_k_final=5
)

# QA Chain 생성 (Cross-Encoder Retriever 사용)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=cross_encoder_retriever,
    return_source_documents=True)

In [89]:

input_text = "가상화폐는 어디에서 사용될 수 있어?"

docs = cross_encoder_retriever.invoke(input_text)
print("\n🔹 ReRanker Retriever 결과:")
for d in docs:
    print("-", d.page_content[:120].replace("\n"," ",), d.metadata)
  


🔹 ReRanker Retriever 결과:
- 20 경제금융용어 700선 계좌대체 중앙예탁기관에 계좌를 설정한 계좌 설정자간의 유가증권 수수를 실물증권의 인수도  대신 계좌간 대체하는 방식으로 장부상 처리하는 것을 말한다. 주식․채권 등 증권거래의  결과 매도자 {'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'creator': 'Smart PDF Imposition 1.0', 'producer': 'Smart PDF Imposition 1.0', 'page_label': '37', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'page': 36}
- 325 ㅎ  헤지펀드 개인을 대상으로 모집한 자금을 고수익 또는 위험회피 등을 목적으로 국제증권시장이 나 국제외환시장에 투자하는 사적투자조합 또는 투자계약을 말한다. 헤지란 본래 위험을  회피, 분산시킨다는 의미이지 {'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'creationdate': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'creationdate--text': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'moddate--text': 'Ja

In [78]:
def get_chatbot_response(input_text):
    print(f"\n🔍 검색 및 재랭킹 과정:")
    print(f"질문: {input_text}")
    
    # Cross-Encoder Retriever로 문서 검색 및 재랭킹
    relevant_docs = cross_encoder_retriever._get_relevant_documents(input_text)
    print(f"재랭킹 후 선택된 문서 수: {len(relevant_docs)}")
    
    # QA Chain 실행
    chatbot_response = qa_chain(input_text)
    return chatbot_response['result'].strip()

In [82]:
# Cross-Encoder Retriever 테스트
test_query = "가계부실위험지수에 대해 설명해"
print(f"테스트 질문: {test_query}")
result = get_chatbot_response(test_query)
print(f"\n답변: {result}")

테스트 질문: 가계부실위험지수에 대해 설명해

🔍 검색 및 재랭킹 과정:
질문: 가계부실위험지수에 대해 설명해
재랭킹 후 선택된 문서 수: 5

답변: 가계부실위험지수(HDRI)는 가구의 소득 흐름뿐만 아니라 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실위험을 평가하는 지표야. 이 지수는 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상환비율(DSR)과 자산 측면에서 평가하는 부채/자산비율(DTA)을 결합하여 산출해. 가계부실위험지수는 가구의 DSR과 DTA가 각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있어. 이 지수가 100을 초과하는 가구를 '위험가구'로 분류하고, 위험가구는 소득 및 자산 측면에서 모두 취약한 '고위험가구', 자산 측면에서 취약한 '고DTA가구', 소득 측면에서 취약한 '고DSR가구'로 구분할 수 있어. 하지만 이 지수는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이지, 이들 가구가 당장 채무상환 불이행에 직면한 것을 의미하지는 않아.


In [84]:
input_text = "경제성장율에 대해서 궁금하당~"
result = get_chatbot_response(input_text)
print(result)


🔍 검색 및 재랭킹 과정:
질문: 경제성장율에 대해서 궁금하당~
재랭킹 후 선택된 문서 수: 5
경제성장률은 일정 기간 동안 한 나라의 경제 규모, 즉 국민소득 규모가 얼마나 늘어났는지를 백분율로 나타낸 거야. 주로 실질국내총생산(실질GDP)의 증가율을 의미하지. 실질GDP는 각 경제활동 부문에서 창출된 실질 부가가치의 합계로 계산돼. 경제성장률을 계산할 때는 전년도의 실질GDP와 금년도의 실질GDP를 비교해서 구해. 여러 해 동안의 연평균 경제성장률을 계산할 때는 복리 연율의 산식을 사용해. 이게 경제성장률에 대한 기본적인 설명이야!


In [88]:
from langchain_core.tracers.stdout import ConsoleCallbackHandler

# 인터페이스 생성
with gr.Blocks() as demo:
    # 챗봇 레이블을 좌측 상단에 구성
    chatbot = gr.Chatbot(label="경제금융용어 챗봇")
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message, 
                        callbacks=[ConsoleCallbackHandler()])
      bot_message = result['result']

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가
      chat_history.append([message, bot_message])  # tuple 대신 list 사용
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
    clear.click(lambda: [], None, chatbot, queue=False)  # 빈 리스트 반환

# 인터페이스 실행
demo.launch(debug=True)

/var/folders/xn/w4kqb1715fz7h49tp_h7hbmr0000gn/T/ipykernel_30643/931656514.py:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="경제금융용어 챗봇")


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "가상화폐는 어디에서 사용될 수 있어?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "가상화폐는 어디에서 사용될 수 있어?",
  "context": "20\n경제금융용어 700선\n계좌대체\n중앙예탁기관에 계좌를 설정한 계좌 설정자간의 유가증권 수수를 실물증권의 인수도 \n대신 계좌간 대체하는 방식으로 장부상 처리하는 것을 말한다. 주식․채권 등 증권거래의 \n결과 매도자의 증권을 매수자에게 이전해야 한다. 그런데 증권의 이전을 위한 실물증권\n의 인도 방식은 자본시장에서 이루어지는 증권의 대량거래와 잦은 소유자 변동 등을 \n감안하면 사실상 불가능하다. 따라서 대부분의 국가에서는 중앙예탁기관의 계좌대체 \n방식으로 증권의 소유권을 이전하고 있으며 우리나라도 한국예탁결제원이 계좌간 대체 \n방식으로 증권결제를 수행하고 있다. 계좌대체는 예탁자의 개별적 청구 또는 시장 참가\n자간의 사전 약정에 따른 결제기관의 청구에 의해 이루어질 수 있다. 우리나라 ｢상법｣에\n서는 증권거래에 따른 증권의 소유권 이전을 위해 해당증권을 교부하도록 하고 있고 , \n｢자본시장법｣에서 예탁자계좌부에 증권의 양도를 목적으로 계좌간 대체의 기재를 한 \n경우 증권의 교부가 있었던 것으로 본다는 교부의제 규정을 두고 있다 . \n 연관검색어 : 중앙예탁기관\n고객확인절차(KYC)\n고객확인절차(KYC; Know Your Customer)는 고객의 신원을 식별하고 확인하는\n(verif

In [98]:
demo.close()

Closing server running on port: 7860
